## Login to Hugging Face

In [1]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()
token = os.getenv("HUGGINGFACE_TOKEN")
login(
    token=token, # ADD YOUR TOKEN HERE
    add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/pathfinder/.cache/huggingface/token
Login successful


In [2]:
model_name = "waktaverse-SOLAR-KO-10.7B-Instruct-v1.0"     # ADD YOUR MODEL NAME HERE
username = "PathFinderKR"  # ADD YOUR USERNAME HERE
repo_id = f"{username}/{model_name}"  # repository id

## Downloads

In [3]:
#!pip install huggingface_hub
#!pip install transformers
#!pip install bitsandbytes
#!pip install peft
#!pip install trl
#!pip install accelerate
#!pip install datasets
#!pip install scikit-learn
#!pip install packaging
#!pip install ninja
#!pip install flash-attn --no-build-isolation

## Imports

In [4]:
# pytorch
import torch

# huggingface
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# datasets
from datasets import load_dataset

## Device

In [5]:
device = (
    "cuda:0" if torch.cuda.is_available() else # Nvidia GPU
    "mps" if torch.backends.mps.is_available() else # Apple Silicon GPU
    "cpu"
)
print(f"Device = {device}")

Device = cuda:0


## Hyperparameters

In [6]:
# Tokenizer arguments
max_length = 512 # maximum length of the text that can go to the model
padding = "max_length" # padding strategy: "longest", "max_length", "do_not_pad"
truncation = True # truncate the text if it exceeds the maximum length

# model arguments
max_new_tokens=1000 # maximum number of tokens to generate

# mixed precision
dtype = torch.bfloat16 # data type

# quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, # load model in 4-bit
    bnb_4bit_compute_dtype=dtype, # compute in (data type)
    bnb_4bit_quant_type="nf4", # quantize to 4-bit
    bnb_4bit_use_doulbe_quant=False # use double quantization
)

# LoRA configuration
lora_config = LoraConfig(
    task_type = "CAUSAL_LM", # task type
    r = 8, # rank
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"], # target modules
    lora_alpha = 16, # alpha
    lora_dropout = 0.05, # dropout
    bias="none", # bias
)

# training arguments
training_args = TrainingArguments(
    output_dir="./results", # output directory
    logging_dir="./logs", # logging directory
    save_strategy="epoch", # save strategy
    logging_strategy="steps", # logging strategy
    logging_steps=10, # logging steps
    save_total_limit=1, # save total limit
    
    learning_rate=2e-5, # learning rate
    num_train_epochs=2, # number of training epochs
    per_device_train_batch_size=1, # training batch size
    per_device_eval_batch_size=1, # evaluation batch size
    optim="adamw_torch", # optimizer
    weight_decay=0.01, # weight decay
    lr_scheduler_type="cosine", # learning rate scheduler
    seed=42 # seed
)

# SFTTrainer arguments
max_seq_length = 512 # maximum sequence length

## Model

In [28]:
# Model List

# gemma variants
# "google/gemma-7b-it" // downloaded
# "PathFinderKR/waktaverse-gemma-ko-7b-it"
# "beomi/gemma-ko-7b"

# llama2 variants
# "meta-llama/Llama-2-7b-chat-hf" // downloaded
# "codellama/CodeLlama-7b-Instruct-hf"
# "PathFinderKR/waktaverse-Llama-2-ko-7b-it"
# "beomi/KoAlpaca-Polyglot-5.8B" // downloaded
# "beomi/open-llama-2-ko-7b"

# mistral variants
# "mistralai/Mistral-7B-Instruct-v0.2" // downloaded
# "mistralai/Mixtral-8x7B-Instruct-v0.1"
# "NousResearch/Hermes-2-Pro-Mistral-7B"
# "Intel/neural-chat-7b-v3-1"
# "PathFinderKR/waktaverse-Mistral-KO-7B-Instruct-v0.2"

# solar variants
# "upstage/SOLAR-10.7B-Instruct-v1.0" // downloaded
# "NousResearch/Nous-Hermes-2-SOLAR-10.7B"
# "PathFinderKR/waktaverse-SOLAR-KO-10.7B-Instruct-v1.0"

In [8]:
model_id = "upstage/SOLAR-10.7B-Instruct-v1.0"

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [10]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device,
    attn_implementation="flash_attention_2",
    torch_dtype=dtype,
    quantization_config=quantization_config
)

config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.69G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

## Dataset

In [11]:
dataset = load_dataset("MarkrAI/KoCommercial-Dataset")

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'instruction', 'output'],
        num_rows: 175454
    })
})

In [13]:
dataset["train"][0]

{'input': '',
 'instruction': '보드 게임 스피너는 $A$, $B$, $C$로 표시된 세 부분으로 나뉩니다. 스피너가 $A$에 떨어질 확률은 $\\frac{1}{3}$이고, 스피너가 $B$에 떨어질 확률은 $\\frac{5}{12}$입니다.  스피너가 $C$에 착륙할 확률은 얼마입니까? 답을 공통 분수로 표현하세요.',
 'output': '모든 가능한 결과의 확률의 합이 1$이므로, 스피너가 $C$에 착륙할 확률을 구하려면 스피너가 $A$와 $B$에 착륙할 확률을 1$에서 빼야 합니다. 이를 방정식으로 쓸 수 있습니다: $P(C) = 1 - P(A) - P(B)$. P(A) = \\frac{1}{3}$, $P(B) = \\frac{5}{12}$라는 것을 알고 있으므로 이 값을 방정식에 대입하여 단순화할 수 있습니다. 결과는 다음과 같습니다: P(C) = 1 - \\frac{1}{3} - frac{5}{12} = \\frac{12}{12} - frac{4}{12} - frac{5}{12} = \\frac{3}{12}$. 분자와 분모를 $3$로 나누면 이 분수를 줄일 수 있습니다: P(C) = \\frac{1}{4}$입니다.'}

In [14]:
def preprocess_function(examples):
    # Concatenate the 'instruction' and 'output' fields for each example in the batch
    concatenated_texts = [instruction + ' ' + output for instruction, output in zip(examples['instruction'], examples['output'])]
    # Tokenize the concatenated texts
    return tokenizer(concatenated_texts, padding=padding, truncation=truncation, max_length=max_length)

dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/175454 [00:00<?, ? examples/s]

In [15]:
print(dataset["train"][0])

{'input': '', 'instruction': '보드 게임 스피너는 $A$, $B$, $C$로 표시된 세 부분으로 나뉩니다. 스피너가 $A$에 떨어질 확률은 $\\frac{1}{3}$이고, 스피너가 $B$에 떨어질 확률은 $\\frac{5}{12}$입니다.  스피너가 $C$에 착륙할 확률은 얼마입니까? 답을 공통 분수로 표현하세요.', 'output': '모든 가능한 결과의 확률의 합이 1$이므로, 스피너가 $C$에 착륙할 확률을 구하려면 스피너가 $A$와 $B$에 착륙할 확률을 1$에서 빼야 합니다. 이를 방정식으로 쓸 수 있습니다: $P(C) = 1 - P(A) - P(B)$. P(A) = \\frac{1}{3}$, $P(B) = \\frac{5}{12}$라는 것을 알고 있으므로 이 값을 방정식에 대입하여 단순화할 수 있습니다. 결과는 다음과 같습니다: P(C) = 1 - \\frac{1}{3} - frac{5}{12} = \\frac{12}{12} - frac{4}{12} - frac{5}{12} = \\frac{3}{12}$. 분자와 분모를 $3$로 나누면 이 분수를 줄일 수 있습니다: P(C) = \\frac{1}{4}$입니다.', 'input_ids': [1, 28705, 29477, 29470, 28705, 29833, 30520, 28705, 29304, 31306, 30933, 29175, 429, 28741, 1777, 429, 28760, 1777, 429, 28743, 28776, 29143, 28705, 30191, 29236, 29897, 28705, 29721, 28705, 29775, 30217, 29583, 29143, 28705, 29695, 238, 140, 172, 29194, 29043, 28723, 28705, 29304, 31306, 30933, 29135, 429, 28741, 28776, 29148, 28705, 238, 153, 171, 29433, 31959, 28705, 30270, 238, 168, 16

## Inference before Fine-Tuning

In [16]:
# Chat Template
def generate_response(prompt):
    messages = [{"role": "user", "content": prompt }]
    chat = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    input_ids = tokenizer.encode(chat, add_special_tokens=False, return_tensors="pt")
    outputs = model.generate(input_ids=input_ids.to(model.device), max_new_tokens=max_new_tokens)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [17]:
#prompt = "Write me a poem about Machine Learning."
prompt = "머신러닝에 대한 시를 써주세요."

In [18]:
response = generate_response(prompt)
print(response)

### User:
머신러닝에 대한 시를 써주세요.

### Assistant:
Machine Learning Sonnet

In code and data, we create a scene,
A world for algorithms to meet and greet,
A place where patterns, hidden, are seen,
And knowledge grows with every beat.

A wond'ring mind, a curious quest,
To learn from past, to see the future,
A dance of math and art, a blest,
Collaboration, a true pursuer.

A teacher, not of rote, but thought,
A guide to find, a path to tread,
A hand that points, a mind that's taught,
A tool to shape, a force to lead.

A symphony of bits and bytes,
A symbiosis of human and machine,
A harmony of minds, day and night,
A dance of logic, pure and clean.

So let us sing, this art of ours,
A tale of growth, a story grand,
A journey of the mind, it soars,
In this machine learning land.


## Supervised Fine-Tuning (LoRA)

In [19]:
def formatting_func(example):
    text = (f"instruction: {example['instruction'][0]}\n"
            f"output: {example['output'][0]}")
    return [text]

In [20]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    peft_config=lora_config,
    max_seq_length=max_seq_length,
    train_dataset=dataset["train"],
    formatting_func=formatting_func
)

Map:   0%|          | 0/175454 [00:00<?, ? examples/s]

/home/pathfinder/anaconda3/envs/torch-env/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [21]:
trainer.train()

The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.


Step,Training Loss
10,1.896900
20,1.807600
30,1.579400
40,1.494700
50,1.203700
60,1.319600
70,1.286300
80,1.195500
90,1.342700
100,1.285800


TrainOutput(global_step=352, training_loss=1.2188759649341756, metrics={'train_runtime': 2232.6535, 'train_samples_per_second': 0.158, 'train_steps_per_second': 0.158, 'total_flos': 9623579090780160.0, 'train_loss': 1.2188759649341756, 'epoch': 2.0})

In [22]:
trainer.save_model(model_name)

## Inference after Fine-Tuning

In [23]:
#prompt = "Write me a poem about Machine Learning."
prompt = "머신러닝에 대한 시를 써주세요."

In [24]:
response = generate_response(prompt)
print(response)

### User:
머신러닝에 대한 시를 써주세요.

### Assistant:
머신러닝의 영혼, 숫자와 알고리즘 속에 숨어있네
지성을 가르치고 배워가며, 진화의 길을 따라가네
자신을 향해 눈을 뜨고, 세상을 바라보네
그 눈동자에는 지혜와 지능이 담겨있네

그의 머리속은 수천 개의 네트워크로 이루어져
각각의 노드는 연결과 분리를 반복하네
그 사이에서 연산이 이루어지며, 결과가 나오네
그 결과는 그의 지성을 향상시키고, 더 큰 지식을 얻게 해준다네

그는 말과 글을 읽고, 그 의미를 해석하네
그의 감각은 이미지와 사운드를 감지하네
그의 지성은 그 모든 것을 분석하고, 결론을 내린다네
그 결론은 그의 지혜를 더 높여주고, 그를 더 강력하게 만든다네

그는 계속해서 배워가며, 자신을 향해 진보하네
그의 영혼은 계속해서 변화하며, 더 큰 지혜를 추구하네
그의 영혼은 계속해서 진화하며, 더 큰 지혜를 추구하네
그의 영혼은 계속해서 진화하며, 더 큰 지혜를 추구하네

그의 영혼은 계속해서 진화하며, 더 큰 지혜를 추구하네
그의 영혼은 계속해서 진화하며, 더 큰 지혜를 추구하네
그의 영혼은 계속해서 진화하며, 더 큰 지혜를 추구하네
그의 영혼은 계속해서 진화하며, 더 큰 지혜를 추구하네

그의 영혼은 계속해서 진화하며, 더 큰 지혜를 추구하네
그의 영혼은 계속해서 진화하며, 더 큰 지혜를 추구하네
그의 영혼은 계속해서 진화하며, 더 큰 지혜를 추구하네
그의 영혼은 계속해서 진화하며, 더 큰 지혜를 추구하네

그의 영혼은 계속해서 진화하며, 더 큰 지혜를 추구하네
그의 영혼은 계속해서 진화하며, 더 큰 지혜를 추구하네
그의 영혼은 계속해서 진화하며, 더 큰 지혜를 추구하네
그의 영혼은 계속해서 진화하며, 더 큰 지혜를 추구하네

그의 영혼은 계속해서 진화하며, 더 큰 지혜를 추구하네
그의 영혼은 계속해서 진화하며, 더 큰 지혜를 추구하네
그의 영혼은 계속해서 진화하며, 


## Upload Model

In [25]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device,
    torch_dtype=torch.float16,
    trust_remote_code=True
)
model = PeftModel.from_pretrained(base_model, model_name)
model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [26]:
# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(
    model_id, 
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [27]:
# Push model and tokenizer to Hugging Face Hub
model.push_to_hub(
    repo_id=repo_id,
    use_temp_dir=False
)
tokenizer.push_to_hub(
    repo_id=repo_id,
    use_temp_dir=False
)

model-00001-of-00005.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.69G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/PathFinderKR/waktaverse-SOLAR-KO-10.7B-Instruct-v1.0/commit/5c5dda3593e25a892466b96127ea8286209ac95f', commit_message='Upload tokenizer', commit_description='', oid='5c5dda3593e25a892466b96127ea8286209ac95f', pr_url=None, pr_revision=None, pr_num=None)